# How many safe restrooms are there for transgender, intersex, and gender nonconforming individuals in the top 50 US cities? 

From https://www.refugerestrooms.org/about:

"What is REFUGE restrooms?

Refuge Restrooms is a web application that seeks to provide safe restroom access for transgender, intersex, and gender nonconforming individuals. Users can search for restrooms by proximity to a search location, add new restroom listings, as well as comment and rate existing listings.

We’re trans led and seek to create a community focused not only on finding existing safe restroom access, but also advocating for transgender, intersex, and gender nonconforming people’s safety.

Where did you get all this data?

The first 4500 entries are thanks to the old Safe2Pee database. The rest of our database is generated by our users. If you know of a gender neutral or safe restroom, please add it to our database!"

Refuge Restrooms has a public API: https://www.refugerestrooms.org/api/docs/ 
The next cell begins the process of building a URl with an API request and culminating in the last cell which imports queries for the 50 most populous US cities to a CSV file.

#### Step 1: Build a URL that gets API requests

In [ ]:
import requests, json

# Connect to the Refuge Restrooms API by building encoded GET request
# API Endpoint: "front door of our API server"

def build_url(page, sterm):
    '''
    Composes an API request URL to the Refuge Restrooms database given a search term
    '''
    API_ENDPOINT = "https://www.refugerestrooms.org/api"
    sterm_part = ""
    # Use Python string formatting to create injection points with %s
    # Python will be expecting a tuple following this string which contains
    # a string value for each of the %s found in base string
    # entered "false" for both ada_accessible and unisex categories - this turns off a filter, which means
    # we will get both "true" and "false" outputs for various restrooms
    sterm_part = "/v1/restrooms/search?page=%s&per_page=100&offset=0&ada=false&unisex=false&query=%s" % (page, sterm)
    full_url = API_ENDPOINT + sterm_part
    print("URL: ", full_url)
    return full_url

#### Step 2: Make an API Request with the URL built

In [ ]:
def make_api_req(url):
    
    # send the request over the network and store response in resp
    resp = requests.get(url)
    print("Made request, response status: ", resp.status_code)
    # check for status code 200
    if(int(resp.status_code) == 200):
        payload_objects = json.loads(resp.text)
        return payload_objects
    else:
        return None # can also return an empty dictionary if you want

#### Step 3: Set up a function that allows us to query by a search term, e.g. a city's name

In [ ]:
def query_by_term(page, sterm):
    results = make_api_req(build_url(page, sterm))
    for i in results:
        if results == []:
            break
    return results

#### Step 4: Set up a function that allows us to pass multiple pages of entries for a city and return results

In [ ]:
def query_by_city(city):
    # master list 
    city_total = []
    # starts at 1 because you'll always make at least one call
    page_counter = 1 # this is the default page number, always asking for page 1
    city_partial = query_by_term(page_counter, city)
    # the city total empty list is appended to include each page that shows up in the query
    city_total.append(city_partial)
    # as long as we have at least one result on a page, we will keep going 
    while len(city_partial) > 0:
        page_counter+=1
        city_partial = query_by_term(page_counter, city)
        city_total.append(city_partial)
    return city_total

#### Step 5: Make a list of top 50 US cities from Wikipedia that can be passed through the query_by_city function

In [ ]:
from csv import DictReader

rr_dict = {}
with open('wiki_top_50_cities.csv', newline='') as csvfile:
    citiesreader = DictReader(csvfile)
    for record in citiesreader:
        del record['2019 rank']
        del record['State']
        del record['Change']
        del record['2010 Census']
        del record['2016 land area']
        del record['2016 population density']
        del record['Location']
        del record['']
        if record['City'] not in rr_dict:
            rr_dict[record['City']] = record['2019 estimate']
print(rr_dict)
# output is a dictionary with the cities and their populations

##### Making the dictionary above into a list of the cities below for the purpose of passing through the query_by_city function

In [ ]:
cities_list = list(rr_dict.keys())
print(cities_list)

### Step 6: Use the functions created above to build a large data structure (i.e. a list of dictionaries) of all 50 cities and their accompanying information

In [ ]:
new_list =[]
for i in range(0, 50):
    results = query_by_city(cities_list[i])
    new_list.append(results)
print(new_list)

##### Here's our big list! It's turned out to be a list of lists of dictionaries, which needs to be "flattened" into simply a list of dictionaries...

In [ ]:
print(new_list)

#### Step 7: Make a list of dictionaries using a list comprehension to flatten the messy list of lists of dictionaries output above

Okay, let's use a list comprehension twice to make a list of dictionaries:

In [ ]:
# https://coderwall.com/p/rcmaea/flatten-a-list-of-lists-in-one-line-in-python
# Someone had this neat idea to flatten a list of lists using a list comprehension!
#print(new_list)
flattened_list = [y for x in new_list for y in x]
#print(flattened_list)
flattened_list2 = [y for x in flattened_list for y in x]
#print(flattened_list2)
#this will get me my total entries count
count = 0
for item in flattened_list2:
    print(item)
    count = count + 1
    print("Running count: ", count)
print(count)

Look at that! 11269 entries pulled from the Refuge Restrooms DB when we query the top 50 US cities!

##### Step 8: Import results to a CSV that can then be cleaned up and inserted into a DB (Postgres)

In [ ]:
from csv import DictWriter

with open('restroomstext.csv', mode = 'w') as restrooms:
    # before for loop set up DictWriter that knows how to append dictionaries in csv format to a file
        headers = flattened_list2[0].keys()
        restroomwriter = DictWriter(restrooms, fieldnames = headers, delimiter = ',')
        restroomwriter.writeheader()
        for record in flattened_list2:
            restroomwriter.writerow(record)

## Switching to a smaller data set to avoid less messy entries 

Unfortunately, upon review of the import of 11269 entries into a CSV, I noticed that for the top 50 US cities, various entries for cities that were entered by users included less precise matches including cities not capitazlied e.g. "philadelphia", or including other smaller towns/cities that may be within a greater CSA but are not one of the top 50 cities outlined in my cities list above. A *substantial* amount of time would be needed to devote to cleaning up the data for accurate stats. 

With that big obstacle in mind, I decided for time's sake to scrap the top 50 cities, and I opted to compare three mid-sized cities of comparable population sizes - Cleveland (pop: 385282), Cincinnati (pop: 301394), and Pittsburgh (302205). I repeated the steps above applied to these three cities instead. 

I still included the option of looking at counts for the top 50 cities in my Postgresql DB that connects to Python, but the entries are reduced to 7735 entries from the orginal 11269 entries retrieved due to a crude clean-up using a filter to strictly exclude all entries that are not spelled exactly as the cities are spelled in my list.

## Repeating my steps applied to the cities of Cleveland, Cinciannati, and Pittsburgh

In [ ]:
import requests, json

# Connect to the Refuge Restrooms API by building encoded GET request
# API Endpoint: "front door of our API server"

def build_url(page, sterm):
    '''
    Composes an API request URL to the Refuge Restrooms database given a search term
    '''
    API_ENDPOINT = "https://www.refugerestrooms.org/api"
    sterm_part = ""
    # Use Python string formatting to create injection points with %s
    # Python will be expecting a tuple following this string which contains
    # a string value for each of the %s found in base string
    sterm_part = "/v1/restrooms/search?page=%s&per_page=100&offset=0&ada=false&unisex=false&query=%s" % (page, sterm)
    full_url = API_ENDPOINT + sterm_part
    print("URL: ", full_url)
    return full_url

In [ ]:
def make_api_req(url):
    
    # send the request over the network and store response in resp
    resp = requests.get(url)
    print("Made request, response status: ", resp.status_code)
    # check for status code 200
    if(int(resp.status_code) == 200):
        payload_objects = json.loads(resp.text)
        return payload_objects
    else:
        return None # can also return an empty dictionary if you want

In [ ]:
def query_by_term(page, sterm):
    results = make_api_req(build_url(page, sterm))
    for i in results:
        if results == []:
            break
    return results

In [ ]:
def query_by_city(city):
    # master list 
    city_total = []
    # starts at 1 because you'll always make at least one call
    page_counter = 1 # this is the default page number, always asking for page 1
    city_partial = query_by_term(page_counter, city)
    # the city total empty list is appended to include each page that shows up in the query
    city_total.append(city_partial)
    # as long as we have at least one result on a page, we will keep going 
    while len(city_partial) > 0:
        page_counter+=1
        city_partial = query_by_term(page_counter, city)
        city_total.append(city_partial)
    return city_total

In [ ]:
three_cities_list = ['Cleveland', 'Cincinnati', 'Pittsburgh']

In [ ]:
new_list =[]
for i in range(0, 3):
    results = query_by_city(three_cities_list[i])
    new_list.append(results)
print(new_list)

In [ ]:
flattened_list = [y for x in new_list for y in x]
flattened_list2 = [y for x in flattened_list for y in x]
count = 0
for item in flattened_list2:
    print(item)
    count = count + 1
    print("Running count: ", count)
print(count)

In [ ]:
from csv import DictWriter

with open('restrooms3cities.csv', mode = 'w') as restrooms:
    # before for loop set up DictWriter that knows how to append dictionaries in csv format to a file
        headers = flattened_list2[0].keys()
        restroomwriter = DictWriter(restrooms, fieldnames = headers, delimiter = ',')
        restroomwriter.writeheader()
        for record in flattened_list2:
            restroomwriter.writerow(record)

## Other inquiry questions that could be explored if I had more time:

##### How many entries per major city?
    # How many entries for unisex bathrooms for each top major city?
    # How many entries for accessible bathrooms for each top major city?
    # What's the proportion of unisex bathrooms to all bathrooms?
    # What's the proportion of accessible bathrooms to all bathrooms?
    # What's the proportion of gender neutral AND accessible BR to all BR?
    # What's the number of restrooms per capita for each top 50 city?
    # How many entries were entered by year?
##### Are there cities in which users tend to provide more descriptive details?
    # Do some cities have more comments than others?
    # Do some cities provide more directions than others?
##### In which major cities is RR utilized the most?
    # What is the proportion of RR utilization to a city's population?
    # What is the proportion of RR utilization to the city's LGBTQ population?
##### How many bathroom entries in each city mention the need to buy or purchase something for bathroom access?
##### How many bathrooms have changing tables?

## Most notable areas of technical growth

I learned how to:

1. Automate the "getting" of API requests
2. Import the API data into a large data structure, i.e. list of dictionaries
3. Utilize list comprehensions for modifying a data structure (i.e. list of lists)
4. Create a relational database in Postgresql
5. Connect Postgres to Python using the psycopg utility
6. Create select statements, and more specifically, aggregates with inner joins to do basic math with SQL
7. Use matplotlib for simple visualizations, i.e. vertical bar chart, horizontal bar chart, and pie charts
8. Use Git to commit files

## Areas of greatest struggle...

I would like to get better at:
    
1. Managing more expedient data cleanup for large data sets by better understanding of Python algorithms (as used in the icon project
2. Using an API with a key and hiding the key from the public
3. Incorporating other data sources in order to create a new data set of my own instead of repeating a DB already made by someone else
4. Creating more complicated matplotlib visualizations
5. Using pandas for data crunching
6. Incorporating other topics learned in peer teaching, like modules
7. Using more functionalization/modularity in my code
8. Merging git branches and uploading to Github